In [35]:
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2 
from torchvision.models.detection import fasterrcnn_resnet50_fpn 
import torch
from conf import NUMBER_OF_CLASSES
import datetime
from torchvision.models.detection import ssd300_vgg16, ssd
import cv2
import os
import sys
from conf import *
import itertools
sys.path.append('..')
sys.path.append('../training')

from inference.inference_helper import denormalize_polygon, preprocess_image, inference_filter_prediction
from training.helper import get_test_data_loader
import numpy as np

from torchvision.ops import box_iou
from sklearn.metrics import confusion_matrix
import seaborn as sns
import torch.nn.functional as F

import matplotlib.pyplot as plt


model_path1 = "../models/Good_FasterRcnn_V1_model.pth"
model_path2 = "../models/Good_FasterRcnn_V2_model.pth"
model_path3 = "../models/Good_SSD_model.pth"


def load_model1(num_classes=NUMBER_OF_CLASSES):
    model = fasterrcnn_resnet50_fpn(pretrained=False)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    state_dict= torch.load(model_path1)
    updated_state = {k.replace("module.", ""): v for k,v in state_dict.items()}
    model.load_state_dict(updated_state)
    print(f"MODEL from volume: {model_path1} is loaded successfuly")
    return model

def load_model2(num_classes=NUMBER_OF_CLASSES):
    model = fasterrcnn_resnet50_fpn_v2(pretrained=False)
    in_features = model.roi_heads.box_predictor.cls_score.in_features

    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    state_dict= torch.load(model_path2)
    updated_state = {k.replace("module.", ""): v for k,v in state_dict.items()}
    model.load_state_dict(updated_state)
    print(f"MODEL from volume: {model_path2} is loaded successfuly")
    return model

def get_ssd_detection_model(num_classes=NUMBER_OF_CLASSES):
    ssd_model = ssd300_vgg16(weights=False)
    num_anchors = ssd_model.anchor_generator.num_anchors_per_location()
    out_channels = [512,1024,512,256,256,256]
    # ssd_model.head = ssd.SSDHead(out_channels, num_anchors, num_classes+1)
    state_dict= torch.load(model_path3)
    updated_state = {k.replace("module.", ""): v for k,v in state_dict.items()}
    ssd_model.load_state_dict(updated_state)
    print(f"MODEL from volume: {model_path3} is loaded successfuly")

    return ssd_model


def save_test_img(img, target, prefix):
    # img = img.permute(2,0,1).cpu().numpy()  # Convert to (height, width, channels)
    # img = img.cpu().numpy()  # Convert to (height, width, channels)

    # Save the image with bounding boxes
    if not os.path.exists(os.path.join(os.getcwd(), 'test_output')):
        os.makedirs(os.path.join(os.getcwd(), 'test_output'))
    # cv2.imshow(img)
    img_path = f"./test_output/output_image_{prefix}.png"
    cv2.imwrite(img_path, img)
    return img_path

def plot_boxes(normalized_road_roi_polygon, results, frame):
    """
    Takes a frame and its results as input, and plots the bounding boxes and label on to the frame.
    :param results: contains labels and coordinates predicted by model on the given frame.
    :param frame: Frame which has been scored.
    :return: Frame with bounding boxes and labels ploted on it.
    """
    label_bg_white = (255, 255, 255)
    if len(results) != 0:
        result = results
    # for result in results:
        for box, label,score in itertools.zip_longest(result['boxes'], result['labels'], result["scores"]):
            label = label.item()
            box_color = BOX_COLOR[label]
            x1, x2, x3, x4 = int(box[0].item()), int(box[1].item()), int(box[2].item()), int(box[3].item())
            cv2.rectangle(frame, (x1,x2),(x3,x4), box_color, 2)
            if score is not None:
                cv2.rectangle(frame, (x1, x2-25), (x1+150, x2), label_bg_white, -1)
                label_text = f'{class_to_label(label)}: {score:.2f}'
                cv2.putText(frame, label_text, (x1, x2 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.9, box_color, 2)
    cv2.polylines(frame, [np.array(normalized_road_roi_polygon)], isClosed=True, color=(32, 32, 128), thickness=2)
    return frame


def class_to_label(label):
    return IDX_TO_CLASSES[label]



In [3]:
# model1 = load_model1()
model1 = load_model2()
# model1 = get_ssd_detection_model()

model1.eval()
# model2.eval()
# model3.eval()
model_name = "V2"
original_image_sizes = []

/opt/homebrew/Caskroom/miniforge/base/envs/mlp/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/mlp/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


MODEL from volume: ../models/Good_FasterRcnn_V2_model.pth is loaded successfuly


In [37]:
IOU_THRESHOLD = 0.4

def generate_hist(output, target, label):
    
    output_mask = output['labels'] == label
    target_mask = target['labels'] == label
    output = {k: v[output_mask] for k, v in output.items()}
    target = {k: v[target_mask] for k, v in target.items()}

    ground_truth_boxes = target["boxes"]
    predicted_boxes = output["boxes"]
    if len(predicted_boxes) == 0:
        predicted_boxes = torch.from_numpy(np.zeros_like(ground_truth_boxes))
    print("ground_truth_boxes: ", ground_truth_boxes)
    print("predicted_boxes: ", predicted_boxes)

    iou_matrix = box_iou(ground_truth_boxes, predicted_boxes)
    print("iou_matrix \n", iou_matrix)
    conf_matrix = torch.zeros(len(ground_truth_boxes), len(predicted_boxes))
    conf_matrix[iou_matrix > IOU_THRESHOLD] = 1
    answers_ground_truth = conf_matrix.sum(axis=1)

    print("CONF MATRIX \n", conf_matrix)
    correct_boxes_prediction = answers_ground_truth.sum().item()
    not_predicted_boxes = len(answers_ground_truth) - answers_ground_truth.sum().item()

    answers_prediction = conf_matrix.sum(axis=0)
    excessive_boxes = len(answers_prediction) - answers_prediction.sum().item()
    return (correct_boxes_prediction, excessive_boxes, not_predicted_boxes)

def save_hist(values, title, prefix):
    categories = ["Correct Box Prediction", "Excessive boxes", "Not Predicted Boxes"]
    sns.barplot(x=categories, y=values, palette="viridis", orientation="vertical")
    print("VALUES: ", values)
    # Add labels and title
    plt.title(title)
    plt.savefig(f'./graphs_output/histogram_{prefix}.png')
    plt.close()



dataloader = get_test_data_loader(1, "./real_frames/")
image_dim = (960,1280)
denormalized_road_roi_polygon = denormalize_polygon(image_dim, ROAD_ROI_POLYGON)

for batch_idx, (data, targets,original_images) in enumerate(dataloader, 1):

    frame = plot_boxes(targets[0], torch.clone(original_images[0]).numpy(), denormalized_road_roi_polygon)
    save_test_img(frame, targets, f"model_image")

    break

    with torch.no_grad():
        outputs1 = model1(data)
    outputs1 = inference_filter_prediction(outputs1, denormalized_road_roi_polygon)
    for output, target, original_image in zip(outputs1, targets, original_images): 
        print("labels: ", output["labels"])
        idx = target["idx"]
        if idx == 0:
            idx = "complex"
        elif idx == 1:
            idx = "medium"
        elif idx == 2:
            idx = "simple"

        del target['idx']
        hist_cars = generate_hist(output, target, 4)
        hist_pedestrians = generate_hist(output, target, 3)
        prefix = f"{model_name}_{idx}"

        frame = plot_boxes(denormalized_road_roi_polygon, [output], torch.clone(original_image).numpy())
        save_test_img(frame, output, f"model_{prefix}")
        target["scores"] = [None]
        frame = plot_boxes(denormalized_road_roi_polygon, [target], torch.clone(original_image).numpy())
        save_test_img(frame, output, f"actual_{prefix}")
        
        
        print("Histograms")
        print(hist_cars)
        print(hist_pedestrians)
        save_hist(hist_cars, title=f"Car prediction by Faster Rcnn {model_name}", prefix=f"car_{prefix}")
        save_hist(hist_pedestrians, title=f"Pedestrian prediction by Faster Rcnn {model_name}", prefix=f"pedestrians_{prefix}")



=====[INFO] Get test data loader test_dir:
FOUND INCONSISTENT FILE: _complex.jpg, 
label not found: _complex.txt
FOUND INCONSISTENT FILE: _simple.jpg, 
label not found: _simple.txt
=====[INFO] Got test loader


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices